# 🎭 EMOTION DETECTION - OPTIMIZED FOR REAL-TIME

## 🎯 Objectives:
- **Balanced Performance**: Akurasi seimbang untuk semua kelas (>65%)
- **Real-time Ready**: FPS tinggi untuk webcam detection (>20 FPS)
- **Production Quality**: Stable dan reliable

## 📊 Dataset:
- Training: 20,573 images (5 kelas)
- Test: 5,114 images
- Format: 48x48 grayscale

## 🚀 Strategy:
1. ✅ Focal Loss untuk class imbalance
2. ✅ Enhanced Class Weights
3. ✅ Advanced Augmentation
4. ✅ Deep CNN Architecture
5. ✅ Real-time Optimization

In [1]:
# Cell 1: Setup & Imports
import warnings
warnings.filterwarnings('ignore')

import os
import platform # Tambahkan ini untuk info OS dan arsitektur
import sys      # Tambahkan ini untuk info versi Python
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
from datetime import datetime
import cv2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Pastikan TensorFlow dan Keras sudah terimport
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *

# Set seed untuk reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Tambahkan pengecekan Keras versi lama/baru (Opsional, tapi bagus buat compatibility)
if tf.__version__ >= '2.0.0':
    print("TensorFlow Keras Backend Activated.")
else:
    print("Warning: Older TensorFlow/Keras version detected.")
    
# Output informasi sistem dan versi
print("="*70)
print("🎭 EMOTION DETECTION - OPTIMIZED FOR REAL-TIME")
print("="*70)

# Info Versi Python (sesuai permintaan)
print(f"🐍 Python Version: {sys.version.split(' ')[0]} (Build: {platform.python_build()[0]})")

# Info OS dan Arsitektur (support di segala device/env)
print(f"💻 OS/Arch: {platform.system()} {platform.release()} ({platform.machine()})")

# Info TensorFlow
print(f"🧠 TensorFlow: {tf.__version__}")
# Cek ketersediaan GPU (penting buat perfomance)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"⚡ GPU Status: {len(gpus)} GPU(s) detected.")
    try:
        # Batasi alokasi memori GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Error jika memory growth harus di-set saat startup
        print(f"Runtime Error Setting GPU Memory: {e}")
else:
    print("🐌 GPU Status: No GPU detected. Running on CPU.")

print("✅ Setup & Environment Check Complete!")
print("="*70)

2025-10-07 20:18:53.177143: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 20:18:53.177330: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 20:18:53.209606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow Keras Backend Activated.
🎭 EMOTION DETECTION - OPTIMIZED FOR REAL-TIME
🐍 Python Version: 3.13.7 (Build: main)
💻 OS/Arch: Linux 6.16.3-76061603-generic (x86_64)
🧠 TensorFlow: 2.20.0
🐌 GPU Status: No GPU detected. Running on CPU.
✅ Setup & Environment Check Complete!


2025-10-07 20:18:53.874170: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 20:18:53.874494: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
E0000 00:00:1759843134.037918  120973 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1759843134.059886  120973 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your pla

In [8]:
# Cell 2: Focal Loss Implementation (Updated for Keras/TF 2.x Idioms)

import tensorflow as tf
# K.backend sudah diimpor di Cell 1, tapi kita lebih baik pakai operasi tf langsung

def focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss untuk mengatasi class imbalance dalam multi-class classification.
    Fungsi ini akan fokus pada 'hard examples' (contoh yang sulit diprediksi).
    
    Args:
        gamma (float): Parameter fokus. Nilai > 0 mengurangi bobot 'easy examples'.
        alpha (float): Parameter penyeimbang. Digunakan untuk mengatasi class imbalance.
        
    Returns:
        function: Fungsi loss yang siap digunakan.
    """
    
    # Pastikan alpha adalah tensor konstan untuk operasi
    alpha_tensor = tf.constant(alpha, dtype=tf.float32)
    gamma_tensor = tf.constant(gamma, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        # Konversi y_true ke float32
        y_true = tf.cast(y_true, tf.float32)

        # Amankan prediksi dari log(0)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        # 1. Hitung Cross Entropy
        cross_entropy = -y_true * tf.math.log(y_pred)
        
        # 2. Hitung Focusing Factor (pt)
        # pt = p_t, probabilitas untuk kelas sebenarnya
        # Kita pakai tf.reduce_sum karena y_true harusnya one-hot encoded
        pt = tf.reduce_sum(y_true * y_pred, axis=-1, keepdims=True)
        
        # 3. Hitung Focal Weight: (1 - pt)^gamma
        focal_weight = tf.pow(1.0 - pt, gamma_tensor)
        
        # 4. Hitung Alpha Weight: alpha untuk kelas positif, (1-alpha) untuk negatif
        # y_true adalah 1 atau 0. 
        # Jika y_true=1, alpha_weight = alpha. Jika y_true=0, alpha_weight = 1-alpha
        alpha_weight = y_true * alpha_tensor + (1 - y_true) * (1.0 - alpha_tensor)
        
        # 5. Gabungkan untuk mendapatkan Loss per-element
        loss = focal_weight * alpha_weight * cross_entropy
        
        # 6. Sum (semua kelas) dan Mean (semua batch)
        # Sum untuk semua kelas (axis=-1), lalu Mean untuk semua batch
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))

    return focal_loss_fixed

# Test call dan output
focal_loss_function = focal_loss(gamma=2.0, alpha=0.25)

print("✅ Focal Loss (TF 2.x Style) Implementation Ready")
print(f"   • Gamma: {focal_loss_function.__closure__[1].cell_contents:.1f}")
print(f"   • Alpha: {focal_loss_function.__closure__[0].cell_contents:.2f}")

✅ Focal Loss (TF 2.x Style) Implementation Ready
   • Gamma: 2.0
   • Alpha: 0.25


In [9]:
# Cell 3: Modern Class Weights Calculator using tf.keras.utils.image_dataset_from_directory

from sklearn.utils import class_weight
import numpy as np
import tensorflow as tf 
import os # Tambahkan import os untuk mengontrol logging

def calculate_modern_class_weights(train_path="train", power=1.5, img_size=(48, 48)):
    """
    Menghitung class weights secara otomatis menggunakan tf.keras.utils.image_dataset_from_directory 
    untuk mendapatkan label dan menerapkan optional exponential scaling.
    """
    # Mengatur level log TF agar pesan 'OUT_OF_RANGE' tidak muncul
    # '2' = WARN (hanya tampilkan Warning dan Error, bukan Info)
    original_log_level = os.environ.get('TF_CPP_MIN_LOG_LEVEL')
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
    
    print("\n⚙️  Analyzing Training Data for Class Weights (Modern API)...")
    
    # 1. Gunakan API modern untuk memuat struktur direktori
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_path,
        labels='inferred',
        label_mode='int', 
        image_size=img_size,
        color_mode='grayscale', 
        shuffle=False, 
        interpolation='bilinear',
        batch_size=32 # Tetap pakai batch_size untuk menghindari error 0-D array
    )
    
    # 2. Ekstrak label dari tf.data.Dataset
    # Menggunakan list comprehension untuk mengumpulkan semua label
    y_classes = np.concatenate([y.numpy() for x, y in train_ds], axis=0)
    class_labels = train_ds.class_names 
    
    # 3. Hitung base weights menggunakan sklearn.utils.class_weight
    base_weights_array = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_classes),
        y=y_classes
    )
    
    base_weights = dict(enumerate(base_weights_array))

    # 4. Enhanced/Exponential Scaling
    if power != 1.0:
        print(f"🔥 Applying Enhanced Scaling (power={power})...")
        class_weights = {
            k: v ** power for k, v in base_weights.items()
        }
        # Normalisasi ke skala yang lebih reasonable (misalnya max 10)
        max_weight = max(class_weights.values())
        class_weights = {
            k: (v / max_weight) * 10 for k, v in class_weights.items()
        }
    else:
        class_weights = base_weights

    
    print("\n⚖️  MODERN ROBUST CLASS WEIGHTS GENERATED:")
    print(f"   • Total Samples: {len(y_classes)}")
    
    # Output yang lebih informatif
    for idx, name in enumerate(class_labels):
        weight_val = class_weights.get(idx, 1.0) 
        print(f"   [{idx}] {name:<10}: weight={weight_val:>5.2f}")

    # Mengembalikan log level ke setting awal setelah fungsi selesai
    if original_log_level:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = original_log_level
    else:
        del os.environ['TF_CPP_MIN_LOG_LEVEL']
        
    return class_weights

# Panggil fungsi
enhanced_weights = calculate_modern_class_weights(train_path="train", power=1.5)


⚙️  Analyzing Training Data for Class Weights (Modern API)...
Found 28709 files belonging to 7 classes.
🔥 Applying Enhanced Scaling (power=1.5)...

⚖️  MODERN ROBUST CLASS WEIGHTS GENERATED:
   • Total Samples: 28709
   [0] angry     : weight= 0.36
   [1] disgusted : weight=10.00
   [2] fearful   : weight= 0.35
   [3] happy     : weight= 0.15
   [4] neutral   : weight= 0.26
   [5] sad       : weight= 0.27
   [6] surprised : weight= 0.51


In [11]:
# Cell 4: Advanced Data Augmentation (Quick Fix & Enhanced)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Impor Path dari Cell 1 (pastikan sudah ada)
# Note: Ini adalah Opsi 1 yang lebih kompatibel dengan flow_from_directory

def create_data_generators(train_path="train", val_path="test", batch_size=32, img_size=(48, 48)):
    """
    Advanced augmentation untuk training menggunakan ImageDataGenerator yang Robust.
    Menggunakan ImageDataGenerator karena kode modelmu di Cell 5 sudah siap untuk ini.
    """
    print("\n🚀 Creating Data Generators (Using ImageDataGenerator)...")
    
    # 1. Training Augmentation - AGRESIF
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=25,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2] 
    )
    
    # 2. Validation Generator - Hanya Rescale
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # 3. Training Flow
    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        # TIDAK PERLU classes=[], karena kita mau 7 kelas terdeteksi otomatis
        shuffle=True
    )
    
    # 4. Validation Flow
    val_gen = val_datagen.flow_from_directory(
        val_path,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        # TIDAK PERLU classes=[]
        shuffle=False
    )
    
    print(f"\n📊 Data Generators Created:")
    print(f"   • Total Classes Detected: {train_gen.num_classes}")
    print(f"   • Train Samples: {train_gen.samples:,} images")
    print(f"   • Val Samples: {val_gen.samples:,} images")
    
    return train_gen, val_gen

train_gen, val_gen = create_data_generators(train_path="train", val_path="test", batch_size=32)


🚀 Creating Data Generators (Using ImageDataGenerator)...
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.

📊 Data Generators Created:
   • Total Classes Detected: 7
   • Train Samples: 28,709 images
   • Val Samples: 7,178 images


In [25]:
# Cell 5: Optimized CNN Model (Updated for 7 Classes & Educational Clarity)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
# Pastikan 'focal_loss' sudah didefinisikan di Cell 2

# PERHATIAN: Set num_classes menjadi 7 (berdasarkan data yang terdeteksi di Cell 3)
def create_optimized_emotion_model(num_classes=7, input_shape=(48, 48, 1)):
    """
    Deep CNN yang dioptimasi untuk emotion detection real-time.
    Menggunakan teknik regulasi dan normalisasi canggih.
    
    Args:
        num_classes (int): Jumlah kelas emosi yang akan diprediksi (default 7).
        input_shape (tuple): Dimensi input gambar (default 48, 48, 1 untuk Grayscale).
    """
    print("\n🧠 Creating Optimized Emotion Model (7 Classes)...")
    
    model = Sequential([
        # Lapisan Input
        Input(shape=input_shape, name='Input_48x48_Grayscale'),
        
        # --- Block 1: Feature Extraction A (Filter 64) ---
        Conv2D(64, (3, 3), padding='same', activation='relu', name='Conv_1_1_64'),
        BatchNormalization(name='BN_1_1'), # Menstabilkan dan mempercepat training
        Conv2D(64, (3, 3), padding='same', activation='relu', name='Conv_1_2_64'),
        BatchNormalization(name='BN_1_2'),
        MaxPooling2D((2, 2), name='MaxPool_1'), # Mengurangi dimensi (Feature Reduction)
        Dropout(0.25, name='Dropout_1'), # Regularisasi untuk mencegah Overfitting
        
        # --- Block 2: Feature Extraction B (Filter 128) ---
        Conv2D(128, (3, 3), padding='same', activation='relu', name='Conv_2_1_128'),
        BatchNormalization(name='BN_2_1'),
        Conv2D(128, (3, 3), padding='same', activation='relu', name='Conv_2_2_128'),
        BatchNormalization(name='BN_2_2'),
        MaxPooling2D((2, 2), name='MaxPool_2'),
        Dropout(0.25, name='Dropout_2'),
        
        # --- Block 3: Feature Extraction C (Filter 256) ---
        Conv2D(256, (3, 3), padding='same', activation='relu', name='Conv_3_1_256'),
        BatchNormalization(name='BN_3_1'),
        Conv2D(256, (3, 3), padding='same', activation='relu', name='Conv_3_2_256'),
        BatchNormalization(name='BN_3_2'),
        MaxPooling2D((2, 2), name='MaxPool_3'),
        Dropout(0.25, name='Dropout_3'),
        
        # --- Block 4: High-Level Features & Global Pooling (Filter 512) ---
        Conv2D(512, (3, 3), padding='same', activation='relu', name='Conv_4_1_512'),
        BatchNormalization(name='BN_4_1'),
        GlobalAveragePooling2D(name='GlobalAvgPool'), # Mengubah Feature Maps menjadi vektor tunggal
        Dropout(0.5, name='Dropout_4'),
        
        # --- Classification Head: Fully Connected Layers (MLP) ---
        Dense(512, activation='relu', name='Dense_1_512'),
        BatchNormalization(name='BN_Dense_1'),
        Dropout(0.5, name='Dropout_Dense_1'),
        
        Dense(256, activation='relu', name='Dense_2_256'),
        BatchNormalization(name='BN_Dense_2'),
        Dropout(0.4, name='Dropout_Dense_2'),
        
        # Lapisan Output
        Dense(num_classes, activation='softmax', name='Output_Softmax') # Softmax untuk probabilitas multi-kelas
    ], name='Optimized_Emotion_CNN')
    
    # --- Konfigurasi Kompilasi ---
    model.compile(
        # Optimizer Adam dengan Learning Rate awal yang konservatif
        optimizer=Adam(learning_rate=0.001), 
        # Menggunakan Focal Loss untuk menangani ketidakseimbangan kelas (Class Imbalance)
        loss=focal_loss(gamma=2.0, alpha=0.25), 
        # Metrik utama yang digunakan untuk evaluasi model
        metrics=['accuracy']
    )
    
    model.summary()
    
    print(f"\n✅ Model Created:")
    print(f"   • Num Classes: {num_classes}")
    print(f"   • Parameters: {model.count_params():,}")
    print(f"   • Size (Float32): ~{model.count_params() * 4 / 1024 / 1024:.1f} MB")
    
    return model

# Panggil fungsi, sekarang default-nya 7 kelas
model = create_optimized_emotion_model()


🧠 Creating Optimized Emotion Model (7 Classes)...


Model: "Optimized_Emotion_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv_1_1_64 (Conv2D)            │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_1_1 (BatchNormalization)     │ (None, 48, 48, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv_1_2_64 (Conv2D)            │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_1_2 (BatchNormalization)     │ (None, 48, 48, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool_1 (MaxPooling2D)        │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_1 (Dropout)             │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv_2_1_128 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_2_1 (BatchNormalization)     │ (None, 24, 24, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv_2_2_128 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_2_2 (BatchNormalization)     │ (None, 24, 24, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool_2 (MaxPooling2D)        │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_2 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv_3_1_256 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_3_1 (BatchNormalization)     │ (None, 12, 12, 256)    │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv_3_2_256 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_3_2 (BatchNormalization)     │ (None, 12, 12, 256)    │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool_3 (MaxPooling2D)        │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_3 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv_4_1_512 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_4_1 (BatchNormalization)     │ (None, 6, 6, 512)      │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GlobalAvgPool                   │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_1_512 (Dense)             │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_Dense_1 (BatchNormalization) │ (None, 512)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 2,728,903 (10.41 MB)

 Trainable params: 2,724,551 (10.39 MB)

 Non-trainable params: 4,352 (17.00 KB)


✅ Model Created:
   • Num Classes: 7
   • Parameters: 2,728,903
   • Size (Float32): ~10.4 MB


In [26]:
# Cell 6: Training with Callbacks & Model Load Check (LOGIC FIXED)

import os
import time
import sys # Tambahkan sys untuk exit yang aman jika perlu
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model 
from datetime import datetime 

# --- 1. Konfigurasi ---
# MODEL_FILE TETAP .keras sesuai permintaanmu, tapi hati-hati dengan error load sebelumnya
MODEL_FILE = 'best_emotion_model.keras' 
log_dir = f"logs/optimized_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Dapatkan fungsi focal_loss_fixed yang sudah didefinisikan di Cell 2
focal_loss_fixed = focal_loss(gamma=2.0, alpha=0.25)


# FUNGSI UTAMA UNTUK MENGONTROL ALUR (BEST PRACTICE)
def run_training_or_load_model(model, train_gen, val_gen, enhanced_weights, epochs=35):
    """
    Cek model yang sudah ada, jika ada, muat dan lewati training.
    Jika tidak ada, set up callbacks dan jalankan training baru.
    """
    
    # --- 2. Pengecekan Model Eksisting ---
    if os.path.exists(MODEL_FILE):
        print(f"\n⚠️ Model file '{MODEL_FILE}' found!")
        
        try:
            # Muat model dengan custom_objects yang diperlukan
            loaded_model = load_model(
                MODEL_FILE, 
                custom_objects={'focal_loss_fixed': focal_loss_fixed}
            )
            print("✅ Model loaded successfully. TRAINING DIESKIP.")
            
            loaded_model.summary(line_length=80) 
            print(f"\nModel {MODEL_FILE} siap digunakan.")
            
            # KODE KRUSIAL: Return model yang sudah di-load agar variabel global 'model' terupdate
            # Dan tidak ada kode training di bawahnya yang dijalankan.
            return loaded_model, None 
            
        except Exception as e:
            print(f"❌ Gagal memuat model ({e}). Melanjutkan ke Training Baru.")
            print("💡 Model baru akan dibuat dan dilatih dari awal.")
            # Di sini, kita return model *lama* yang belum dilatih
            return model, None
            
    else:
        # --- 3. Callbacks dan Training Baru ---
        print("🚀 Starting Training...\n")
        
        callbacks = [
            EarlyStopping(
                monitor='val_accuracy',
                patience=12,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            # ModelCheckpoint menyimpan ke format .keras
            ModelCheckpoint(
                MODEL_FILE, 
                monitor='val_accuracy',
                save_best_only=True,
                verbose=1
            ),
            TensorBoard(log_dir=log_dir)
        ]
        
        start_time = time.time()
        
        # --- 4. Training Model ---
        history = model.fit(
            train_gen,
            epochs=epochs,
            validation_data=val_gen,
            class_weight=enhanced_weights, 
            callbacks=callbacks,
            verbose=1
        )
        
        training_time = time.time() - start_time
        best_acc = max(history.history['val_accuracy'])
        
        print(f"\n✅ Training Complete!")
        print(f"   Waktu: {training_time/60:.1f} min")
        print(f"   Akurasi Terbaik: {best_acc:.4f} ({best_acc*100:.2f}%)")

        # Return model setelah training selesai
        return model, history


# --- JALANKAN FUNGSI ---
# Variabel global 'model' akan diperbarui dengan hasil fungsi
model, history = run_training_or_load_model(model, train_gen, val_gen, enhanced_weights)


⚠️ Model file 'best_emotion_model.keras' found!
✅ Model loaded successfully. TRAINING DIESKIP.


Model: "Optimized_Emotion_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                   │ (None, 48, 48, 64)       │           640 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization               │ (None, 48, 48, 64)       │           256 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)                 │ (None, 48, 48, 64)       │        36,928 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_1             │ (None, 48, 48, 64)       │           256 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)      │ (None, 24, 24, 64)       │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout (Dropout)                 │ (None, 24, 24, 64)       │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)                 │ (None, 24, 24, 128)      │        73,856 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_2             │ (None, 24, 24, 128)      │           512 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)                 │ (None, 24, 24, 128)      │       147,584 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_3             │ (None, 24, 24, 128)      │           512 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)    │ (None, 12, 12, 128)      │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_1 (Dropout)               │ (None, 12, 12, 128)      │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)                 │ (None, 12, 12, 256)      │       295,168 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_4             │ (None, 12, 12, 256)      │         1,024 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)                 │ (None, 12, 12, 256)      │       590,080 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_5             │ (None, 12, 12, 256)      │         1,024 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)    │ (None, 6, 6, 256)        │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_2 (Dropout)               │ (None, 6, 6, 256)        │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)                 │ (None, 6, 6, 512)        │     1,180,160 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_6             │ (None, 6, 6, 512)        │         2,048 │
│ (BatchNormalization)              │                          │               │
├──────────────────────────────

 Total params: 8,176,465 (31.19 MB)

 Trainable params: 2,724,037 (10.39 MB)

 Non-trainable params: 4,352 (17.00 KB)

 Optimizer params: 5,448,076 (20.78 MB)


Model best_emotion_model.keras siap digunakan.


In [27]:
# Cell 7: Training with Callbacks (UPDATED dengan Skip Logic)

import os
import time
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from datetime import datetime

# Asumsi model.py sudah mendefinisikan focal_loss

# Definisikan Path Model
MODEL_PATH = 'best_emotion_model.keras'

# Logika Skip Training
if os.path.exists(MODEL_PATH):
    print(f"♻️ Model '{MODEL_PATH}' ditemukan. Melewati proses training.")
    
    # MUAT MODEL LAMA
    try:
        # Kita harus panggil fungsi focal_loss() di sini karena custom_objects butuh fungsi itu sendiri
        # Asumsi focal_loss() didefinisikan di Cell 2
        model = keras.models.load_model(
            MODEL_PATH,
            custom_objects={'focal_loss_fixed': focal_loss(gamma=2.0, alpha=0.25)}
        )
        # Set variabel untuk kompatibilitas Cell 8/9/output ringkasan
        history = None 
        best_acc = model.evaluate(val_gen, verbose=0)[1] # Ambil akurasi validasi model yang sudah ada
        training_time = 0.0
        print(f"✅ Model lama berhasil dimuat. Val Accuracy saat ini: {best_acc:.4f} ({best_acc*100:.2f}%)")
        
    except Exception as e:
        print(f"❌ Error memuat model yang sudah ada: {e}. Akan memulai training baru.")
        # Hapus file yang rusak agar proses training baru bisa berjalan lancar
        os.remove(MODEL_PATH) 
        # Lanjutkan ke blok 'else' dengan re-run cell
        
else:
    if 'train_gen' not in locals() or train_gen is None or 'enhanced_weights' not in locals() or enhanced_weights is None:
        print("❌ Gagal training. Pastikan Cell 3 (Load Data) dan Cell 5 (Data Generators) sudah dijalankan.")
    else:
        print("🚀 Starting Training...\n")
        
        log_dir = f"logs/optimized_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        callbacks = [
            EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
            ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
            TensorBoard(log_dir=log_dir)
        ]
        
        # Perhitungan langkah per epoch eksplisit
        STEPS_PER_EPOCH = train_gen.samples // train_gen.batch_size
        VALIDATION_STEPS = val_gen.samples // val_gen.batch_size
        
        start_time = time.time()
        
        # MULAI TRAINING
        history = model.fit(
            train_gen,
            steps_per_epoch=STEPS_PER_EPOCH,
            epochs=35, 
            validation_data=val_gen,
            validation_steps=VALIDATION_STEPS,
            class_weight=enhanced_weights,
            callbacks=callbacks,
            verbose=1
        )
        
        training_time = time.time() - start_time
        best_acc = max(history.history['val_accuracy'])
        
        print(f"\n✅ Training Complete!")
        print(f"   Time: {training_time/60:.1f} min")
        print(f"   Best Accuracy: {best_acc:.4f} ({best_acc*100:.2f}%)")

♻️ Model 'best_emotion_model.keras' ditemukan. Melewati proses training.
❌ Error memuat model yang sudah ada: Dimensions must be equal, but are 7 and 5 for '{{node compile_loss/focal_loss_fixed/mul}} = Mul[T=DT_FLOAT](compile_loss/focal_loss_fixed/Neg, compile_loss/focal_loss_fixed/Log)' with input shapes: [?,7], [?,5].. Akan memulai training baru.


In [ ]:
# Cell 8: Comprehensive Evaluation

best_model = keras.models.load_model(
    'best_emotion_model.keras',
    custom_objects={'focal_loss_fixed': focal_loss(2.0, 0.25)}
)

print("📈 Evaluating Model...\n")

val_gen.reset()
predictions = best_model.predict(val_gen, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = val_gen.classes

class_names = ['MARAH', 'JIJIK', 'TAKUT', 'SENANG', 'SEDIH']

print("\n📊 CLASSIFICATION REPORT:")
print("="*60)
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

cm = confusion_matrix(y_true, y_pred)

print("\n📊 PER-CLASS ACCURACY:")
print("-"*60)
accuracies = []
for i, name in enumerate(class_names):
    acc = cm[i, i] / cm[i].sum()
    accuracies.append(acc)
    
    status = "✅" if acc >= 0.65 else "⚠️" if acc >= 0.55 else "❌"
    bar = "█" * int(acc * 40)
    print(f"{name:<10}: {acc:.4f} ({acc*100:>5.1f}%) {bar} {status}")

balance_score = min(accuracies) / max(accuracies)
print(f"\n⚖️  Balance Score: {balance_score:.3f}")
print(f"📊 Average Accuracy: {np.mean(accuracies):.4f}")

In [ ]:
# Cell 9: Confusion Matrix Visualization

plt.figure(figsize=(10, 8))

cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Percentage'},
            linewidths=1, linecolor='gray')

plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)

plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrix saved!")

In [ ]:
# Cell 10: Real-time Webcam Detection

def realtime_emotion_detection():
    """
    Real-time emotion detection dengan webcam
    Optimized untuk FPS tinggi
    """
    model = keras.models.load_model(
        'best_emotion_model.keras',
        custom_objects={'focal_loss_fixed': focal_loss(2.0, 0.25)}
    )
    
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )
    
    emotions = ['MARAH', 'JIJIK', 'TAKUT', 'SENANG', 'SEDIH']
    colors = [(0,0,255), (0,255,255), (128,0,128), (0,255,0), (255,0,0)]
    
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("\n📹 Real-time Detection Started")
    print("   Press 'Q' to quit")
    print("   Press 'S' to save screenshot\n")
    
    fps_time = time.time()
    fps = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5,
            minSize=(48, 48), maxSize=(300, 300)
        )
        
        for (x, y, w, h) in faces:
            face = cv2.resize(gray[y:y+h, x:x+w], (48, 48))
            face_input = face.reshape(1, 48, 48, 1) / 255.0
            
            pred = model.predict(face_input, verbose=0)
            idx = np.argmax(pred)
            emotion = emotions[idx]
            confidence = pred[0][idx] * 100
            
            # Draw rectangle dan text
            cv2.rectangle(frame, (x, y), (x+w, y+h), colors[idx], 3)
            
            # Background untuk text
            text = f'{emotion}: {confidence:.1f}%'
            (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(frame, (x, y-th-10), (x+tw+10, y), colors[idx], -1)
            cv2.putText(frame, text, (x+5, y-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        
        # FPS counter
        if time.time() - fps_time > 1:
            fps_time = time.time()
            fps = fps * 0.7 + 0.3 / (time.time() - fps_time)
        
        cv2.putText(frame, f'FPS: {int(fps)}', (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        
        cv2.imshow('Emotion Detection - Press Q to Quit', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            filename = f'screenshot_{datetime.now().strftime("%Y%m%d_%H%M%S")}.jpg'
            cv2.imwrite(filename, frame)
            print(f"   Screenshot saved: {filename}")
    
    cap.release()
    cv2.destroyAllWindows()
    print("\n✅ Session ended")

# Run detection
realtime_emotion_detection()